In [36]:
from IPython.display import clear_output
from langchain_core.prompts import PromptTemplate
from langchain_openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

# 1. Streaming Chat Responses - OpenAI API

In this part, we will only use streaming responses. For OpenAI models, streaming is not yet supported in langchain (Jul 2024). Hence, we will use streaming from their original algorithm. However, langchain supports streaming for other models, so in future, it might also for OpenAI.

In [127]:
# Init client (reads API from environment variable)
client = OpenAI()

In [57]:
# Streaming response
response = client.chat.completions.create(
    model="gpt-3.5-turbo-0125",
    messages=[
        {'role': 'user', 'content': 'Count to 100, with a comma between each number. E.g., 1, 2, 3,'}
    ],
    temperature=0.,
    stream=True
)

In [59]:
# Stream chat response
history = ""
for chunk in response:
    clear_output(wait=False)
    model_output = chunk.choices[0].delta.content
    if model_output:
        history += model_output
    print(history)

# Image generation - OpenAI API

DALL-E-3 is not yet supported by langchain. Hence, we will take a hybrid approach here. We will first generate a prompt with langchain's GPT interface. Then, we will use native OpenAI API for DALL-E-3.

In [112]:
# Get a response of an image description
llm = OpenAI(temperature=0.9)
prompt = PromptTemplate(
    input_variables=["image_desc"],
    template="""
        Generate a detailed prompt to generate an image based on the following description in under 500 words.
        The image should be in a style for a children's book.
        The image description: '''{image_desc}'''
    """,
)
chain = prompt | llm

image_description = chain.invoke(input={"image_desc": "a white siamese cat"})
image_description

In [131]:
prompt.format(image_desc="a white cat")

In [113]:
len(image_description)

In [114]:
# Get a response of a Whisper
response = client.images.generate(
  model="dall-e-3",
  prompt=image_description,
  size="1024x1024",
  quality="standard",  # ["standard", "hd"]
  n=1,
)

image_url = response.data[0].url

In [115]:
image_url

# 3. Text-to-speech - OpenAI API

Keep in mind that OpenAI uses TTS model for this. On the other hand, Whisper is used for speech-to-text.


In [126]:
# Get voice and save to a file
with client.audio.speech.with_streaming_response.create(
    model="tts-1",
    voice="nova",
    input=image_description,
) as response:
    # Save voice
    response.stream_to_file("output.mp3")
    
# Later load as "st.audio(output.mp3)"

4. Speech-to-Text - OpenAI
For this we need to use whisper. We can use the library "openai-whisper". For detailed info, look in project 15.

In [122]:
# TODO: code for whisper